# COVID-19 pandemic Insight
### Data Engineering Capstone Project

#### Project Summary
The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [ ]:
# Import all modules then config&setup Sparksession 
import os
import configparser
import pandas as pd

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, desc, asc, to_date, lit, concat, year, month, dayofmonth
from pyspark.sql.types import StructType, StructField, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date

output_path = './Project_Warehouse/'

spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

### Step 1: Scope the Project and Gather Data

#### Scope 
This Capstone Project's purpose is to preparing data for analysis and finding significant conncections between many aspects the associates with COVID19 pandemic, such as covid19 situation in each countries, accesses to vaccines and effects from COVID19 which are unemployment in many country and trend of world's happiness index 

#### Describe and Gather Data 
The project consists of the following data :
* COVID-19 World Vaccination Progress Data by Our World in Data
* COVID-19 World Vaccine Adverse Reactions(VAERS) by FDA and CDC 
* Unemployment rate by OECD
* World Happiness Report

In [ ]:
#Read all data from prepared dataset
country_covidStatus_path = './World_COVID19_vaccinations_dataset/country_covid_report.csv'
country_vac_by_man_path = './World_COVID19_vaccinations_dataset/country_vaccinations_by_manufacturer.csv'
vac_adverse_patinfo_path = './World_COVID19_vaccine_adverse_reaction/2021VAERSDATA.csv'
vac_adverse_syminfo_path = './World_COVID19_vaccine_adverse_reaction/2021VAERSSYMPTOMS.csv'
vac_adverse_vacinfo_path = './World_COVID19_vaccine_adverse_reaction/2021VAERSVAX.csv'
unemploy_path = './Unemployment_rate_OECD.json'
happiness_2018_report = './World_Happiness_Report/2018_report.csv'
happiness_2019_report = './World_Happiness_Report/2019_report.csv'
happiness_2020_report = './World_Happiness_Report/2020_report.csv'

df_country_covidStatus = spark.read.csv(country_covidStatus_path,header=True)
df_country_vacInfoManufacturer = spark.read.csv(country_vac_by_man_path,header=True)
df_vacAdverse_patInfo = spark.read.csv(vac_adverse_patinfo_path,header=True)
df_vacAdverse_symInfo = spark.read.csv(vac_adverse_syminfo_path,header=True)
df_vacAdverse_vacInfo = spark.read.csv(vac_adverse_vacinfo_path,header=True)
df_unemployRate = spark.read.json(unemploy_path, multiLine=True)
df_happiness_2018 = spark.read.csv(happiness_2018_report,header=True)
df_happiness_2019 = spark.read.csv(happiness_2019_report,header=True)
df_happiness_2020 = spark.read.csv(happiness_2020_report,header=True)

#### COVID-19 World Vaccination Progress Data
The dataset was obtained from Our World in Data : [Coronavirus (COVID-19) Vaccinations](https://ourworldindata.org/covid-vaccinations)

The complete COVID-19 dataset includes all historical data on the pandemic up to the date of publication.

In [ ]:
df_country_covidStatus.limit(5).toPandas()

Country level vaccination data is gathered and assembled in one single file, with manufacturers information, is included. Data shows the vaccines each country received increasing and sorted by date. 

In [ ]:
df_country_vacInfoManufacturer.limit(5).toPandas()

#### COVID-19 World Vaccine Adverse Reactions(VAERS) by FDA and CDC 
This dataset is downloaded from [VAERS](https://vaers.hhs.gov/data/datasets.html?) datasets and for more details on the dataset refer to the [User Guide](https://vaers.hhs.gov/docs/VAERSDataUseGuide_November2020.pdf).

The Vaccine Adverse Event Reporting System (VAERS) was created by the Food and Drug Administration (FDA) and Centers for Disease Control and Prevention (CDC) to receive reports about adverse events that may be associated with vaccines. No prescription drug or biological product, such as a vaccine, is completely free from side effects. Vaccines protect many people from dangerous illnesses, but vaccines, like drugs, can cause side effects, a small percentage of which may be serious. VAERS is used to continually monitor reports to determine whether any vaccine or vaccine lot has a higher than expected rate of events.

In [ ]:
df_vacAdverse_patInfo.limit(5).toPandas()

In [ ]:
df_vacAdverse_symInfo.limit(5).toPandas()

In [ ]:
df_vacAdverse_vacInfo.limit(5).toPandas()

#### Unemployment Rate by OECD
The dataset was obtained from OECD : [Unemployment rate](https://data.oecd.org/unemp/unemployment-rate.htm)

The unemployed are people of working age who are without work, are available for work, and have taken specific steps to find work. The uniform application of this definition results in estimates of unemployment rates that are more internationally comparable than estimates based on national definitions of unemployment. This indicator is measured in numbers of unemployed people as a percentage of the labour force and it is seasonally adjusted. The labour force is defined as the total number of unemployed people plus those in employment. Data are based on labour force surveys (LFS).  For European Union countries where monthly LFS information is not available, the monthly unemployed figures are estimated by Eurostat.

In [ ]:
df_unemployRate.limit(5).toPandas()

#### Unemployment Rate by OECD
The dataset was obtained from Kaggle Dataset : [World Happiness Report](https://www.kaggle.com/yamaerenay/world-happiness-report-preprocessed?select=2020_report.csv)

This dataset contains the happiness score of each country, and some key factors that contribute directly to the overall happiness of the country which are economic production, social support, life expectancy, freedom, generosity, and absence of corruption. The data source has information from 2018 to 2020.

In [ ]:
df_happiness_2018.limit(5).toPandas()

_________________

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.
* Some dataframe have missing values in important column the will be used further
* Every dataframe collect data in string type
* Date format are different in each dataframe
* Vaccine name format are different in each dataframe

#### Cleaning Steps
* Scope and extract intersted column from each dataframe into specified table
* Drop rows with missing values in key columns
* Correct the data type of each columns
* Format the date column
* Format the vaccine name column
* Add date column to each World Happiness Report file and aggregate into a table


In [ ]:
# Determine columns for each table

country_covidStatus_col = ['date','iso_code','location','total_cases','new_cases','total_deaths','new_deaths','total_tests','new_tests','population','population_density','median_age']
country_vacInfoManufacturer_col = ['location','date','vaccine','total_vaccinations']
vac_adverse_vacinfo_col = ['VAERS_ID','VAX_NAME','VAX_MANU','VAX_DOSE_SERIES','VAX_ROUTE','VAX_SITE']
vac_adverse_patinfo_col = ['VAERS_ID','AGE_YRS','SEX','DISABLE','ALLERGIES','VAX_DATE','RECOVD','DIED','DATEDIED','HISTORY']
vac_adverse_syminfo_col = ['VAERS_ID','SYMPTOM1','SYMPTOM2','SYMPTOM3','SYMPTOM4','SYMPTOM5']

# Extract specified columns from each dataframe to table

country_covidStatus_table = df_country_covidStatus.selectExpr(country_covidStatus_col)
country_vacInfoManufacturer_table = df_country_vacInfoManufacturer.selectExpr(country_vacInfoManufacturer_col)
vacAdverse_vacInfo_table = df_vacAdverse_vacInfo.selectExpr(vac_adverse_vacinfo_col)
vacAdverse_patInfo_table = df_vacAdverse_patInfo.selectExpr(vac_adverse_patinfo_col)
vacAdverse_symInfo_table = df_vacAdverse_symInfo.selectExpr(vac_adverse_syminfo_col)

# Drop rows with missing important value

country_covidStatus_table.dropna(how = "any", subset = ["date", "iso_code", "location"])
country_vacInfoManufacturer_table.dropna(how = "any", subset = ["date","location"])
vacAdverse_vacInfo_table.dropna(how = "any", subset = ["VAERS_ID","VAX_MANU"])
vacAdverse_patInfo_table.dropna(how = "any", subset = ["VAERS_ID"])
vacAdverse_symInfo_table.dropna(how = "any", subset = ["VAERS_ID"])

# Correct data type of some table's column

country_covidStatus_table = country_covidStatus_table.withColumn('date',col('date').cast(Date()))
country_covidStatus_table = country_covidStatus_table.withColumn('total_cases',col('total_cases').cast(Int()))
country_covidStatus_table = country_covidStatus_table.withColumn('new_cases',col('new_cases').cast(Int()))
country_covidStatus_table = country_covidStatus_table.withColumn('total_deaths',col('total_deaths').cast(Int()))
country_covidStatus_table = country_covidStatus_table.withColumn('new_deaths',col('new_deaths').cast(Int()))
country_covidStatus_table = country_covidStatus_table.withColumn('total_tests',col('total_tests').cast(Int()))
country_covidStatus_table = country_covidStatus_table.withColumn('new_tests',col('new_tests').cast(Int()))
country_covidStatus_table = country_covidStatus_table.withColumn('population',col('population').cast(Int()))
country_covidStatus_table = country_covidStatus_table.withColumn('population_density',col('population_density').cast(Dbl()))
country_covidStatus_table = country_covidStatus_table.withColumn('median_age',col('median_age').cast(Dbl()))

country_vacInfoManufacturer_table = country_vacInfoManufacturer_table.withColumn('date',col('date').cast(Date()))
country_vacInfoManufacturer_table = country_vacInfoManufacturer_table.withColumn('total_vaccinations',col('total_vaccinations').cast(Int()))

vacAdverse_patInfo_table = vacAdverse_patInfo_table.withColumn('AGE_YRS',col('AGE_YRS').cast(Int()))

unemployRate_table = df_unemployRate.withColumn('Value',col('Value').cast(Dbl()))

happinessSchema = StructType([
    StructField('country',Str()),
    StructField('happiness_score',Dbl()),
    StructField('gdp_per_capita',Dbl()),
    StructField('social_support',Dbl()),
    StructField('health',Dbl()),
    StructField('freedom',Dbl()),
    StructField('generosity',Dbl()),
    StructField('government_trust',Dbl()),
    StructField('continent',Str())
])
happiness_2018_table = spark.read.csv(happiness_2018_report,header=True,sep=',',schema=happinessSchema)
happiness_2019_table = spark.read.csv(happiness_2019_report,header=True,sep=',',schema=happinessSchema)
happiness_2020_table = spark.read.csv(happiness_2020_report,header=True,sep=',',schema=happinessSchema)

# Format the date data

vacAdverse_patInfo_table = vacAdverse_patInfo_table.withColumn('VAX_DATE',to_date(col('VAX_DATE'),'MM/dd/yyyy'))
vacAdverse_patInfo_table = vacAdverse_patInfo_table.withColumn('DATEDIED',to_date(col('DATEDIED'),'MM/dd/yyyy'))

unemployRate_table = unemployRate_table.withColumn('TIME',to_date(col('TIME'),'yyyy-MM'))
unemployRate_table = unemployRate_table.drop(col('Flag Codes'))
unemployRate_table = unemployRate_table.withColumnRenamed("LOCATION", "country_code")

In [ ]:
# Add date column to each World Happiness Report file and integrate into a table

happiness_2018_table = happiness_2018_table.withColumn('date',lit('2018')).withColumn('date',to_date(col('date'),'yyyy'))
happiness_2019_table = happiness_2019_table.withColumn('date',lit('2019')).withColumn('date',to_date(col('date'),'yyyy'))
happiness_2020_table = happiness_2020_table.withColumn('date',lit('2020')).withColumn('date',to_date(col('date'),'yyyy'))
happinessRate_table = happiness_2018_table.union(happiness_2019_table).distinct()
happinessRate_table = happinessRate_table.union(happiness_2020_table).distinct()

In [ ]:
# Check distinct vaccine names in country_vacInfoManufacturer_table

country_vacInfoManufacturer_table.select('vaccine').dropDuplicates().toPandas()

In [ ]:
# Check distinct vaccine names in vacAdverse_vacInfo_table

vacAdverse_vacInfo_table.select('VAX_NAME').dropDuplicates().limit(10).toPandas()

In [ ]:
vacAdverse_vacInfo_table.select('VAX_NAME').dropDuplicates().filter(col("VAX_NAME").contains("COVID")).toPandas()

In [ ]:
# Format vaccine's name and remove all the adverse reaction not from Covid-19 vaccine
vacAdverse_vacInfo_table = vacAdverse_vacInfo_table.where(col('VAX_NAME').contains('COVID19'))
vacAdverse_vacInfo_table = vacAdverse_vacInfo_table.na.replace("COVID19 (COVID19 (UNKNOWN))","Unknown")
vacAdverse_vacInfo_table = vacAdverse_vacInfo_table.na.replace("COVID19 (COVID19 (PFIZER-BIONTECH))","Pfizer/BioNTech")
vacAdverse_vacInfo_table = vacAdverse_vacInfo_table.na.replace("COVID19 (COVID19 (MODERNA))","Moderna")
vacAdverse_vacInfo_table = vacAdverse_vacInfo_table.na.replace("COVID19 (COVID19 (JANSSEN))","Johnson&Johnson")

In [ ]:
# Generate PRIMARY KEY (ID column)for some tables
country_covidStatus_table = country_covidStatus_table.withColumn('ID',concat('date','iso_code'))
country_vacInfoManufacturer_table = country_vacInfoManufacturer_table.withColumn('ID',concat('date','location'))
unemployRate_table = unemployRate_table.withColumn('ID',concat('TIME','country_code'))
happinessRate_table = happinessRate_table.withColumn('ID',concat('date','country'))

### Recheck the schema and data inside

In [ ]:
country_covidStatus_table.printSchema()
country_vacInfoManufacturer_table.printSchema()
vacAdverse_vacInfo_table.printSchema()
vacAdverse_patInfo_table.printSchema()
vacAdverse_symInfo_table.printSchema()
unemployRate_table.printSchema()
happinessRate_table.printSchema()

In [ ]:
country_covidStatus_table.limit(10).toPandas()

In [ ]:
country_vacInfoManufacturer_table.limit(10).toPandas()

In [ ]:
vacAdverse_vacInfo_table.limit(10).toPandas()

In [ ]:
vacAdverse_patInfo_table.limit(10).toPandas()

In [ ]:
vacAdverse_symInfo_table.limit(10).toPandas()

In [ ]:
unemployRate_table.limit(10).toPandas()

In [ ]:
happinessRate_table.limit(10).toPandas()

_________________

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

### Schema:

|          Table name         |                                                                Columns                                                                |                             Description                            |       Type      |
|:---------------------------:|:-------------------------------------------------------------------------------------------------------------------------------------:|:------------------------------------------------------------------:|:---------------:|
|     country_covidStatus     | ID,date,iso_code,location,total_cases,new_cases, total_deaths,new_deaths,total_tests,new_tests, population,population_density,median_age |      stores covid-19 situation and information of each country     |    fact table   |
| country_vacInfoManufacturer |                                                ID,location,date,vaccine,total_vaccinations                                               |            stores covid-19 vaccinations of each country            |    fact table   |
|      vacAdverse_vacInfo     |                                     VAERS_ID,VAX_NAME,VAX_MANU,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE                                     | stores vaccine information of patients who has an adverse reaction | dimension table |
|      vacAdverse_patInfo     |                             VAERS_ID,AGE_YRS,SEX,DISABLE,ALLERGIES,VAX_DATE, RECOVD,DIED,DATEDIED,HISTORY                             |       stores patients information who has an adverse reaction      | dimension table |
|      vacAdverse_symInfo     |                                         VAERS_ID,SYMPTOM1,SYMPTOM2,SYMPTOM3,SYMPTOM4,SYMPTOM5                                         |         stores patients's adverse reaction from the vaccine        | dimension table |
|       unemploymentRate      |                                      ID,country_code,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value                                     |              stores unemployment rate of each country              | dimension table |
|        happinessRate        |            ID,country,happiness_score,gdp_per_capita,social_support, health,freedom,generosity,government_trust,continent,date           |                stores happiness rate of each country               | dimension table |
|  unemploy_CovidStatus |      ID,date,iso_code,location,Value,total_cases,total_deaths,total_tests,population      | stores information about unemployment rate  and COVID-19 situation in each country | fact table |
| happiness_CovidStatus | ID,date,iso_code,location,happiness_score,total_cases,total_deaths,total_tests,population |   stores information about happiness rate  and COVID-19 situation in each country  | fact table |

#### Tables Decision:

As stated above about the objectives of the project that is to find significant connections between diverse aspects about COVID-19 pandemic, So we must have the country_covidStatus and country_vacInfoManufacturer tables to store the key information and then we can integrate them with other aspects information to find data consistency. to do so efficiently, we need identifiers on all tables so they can be joined efficiently such as the date, iso_code, country name, vaccine name etc.

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

1. Reads all the data from dataset using spark
2. Extracts data from datframe and transfroms to described schema
3. Aggregates data to find insight relation between each tables
3. writes the data to destination(in this case is local) in the parquet format.

_________________

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

#### After the prepartion of dataset into formatted schema, next step is to join and insert into remaining tables

In [ ]:
# merge unemployRate_table with country_covidStatus_table

unemploy_CovidStatus_col = ['date','iso_code','location','Value','total_cases','total_deaths','total_tests','population']
unemploy_CovidStatus = unemployRate_table.join(country_covidStatus_table,( (year(col('TIME')) == year(col('date'))) & (month(col('TIME')) == month(col('date'))) & (col('country_code') == col('iso_code')) & (dayofmonth(col('date'))==1) ))
unemploy_CovidStatus = unemploy_CovidStatus.selectExpr(unemploy_CovidStatus_col).withColumnRenamed('Value','unemployed_rate')
unemploy_CovidStatus = unemploy_CovidStatus.withColumn('ID',concat('date','iso_code'))
unemploy_CovidStatus.limit(10).toPandas()

In [ ]:
# merge happinessRate_table with country_covidStatus_table

happiness_CovidStatus_col = ['date','iso_code','location','happiness_score','total_cases','total_deaths','total_tests','population']
happinessRate_table = happinessRate_table.withColumnRenamed('date','Datetime')
happiness_CovidStatus = happinessRate_table.join(country_covidStatus_table,( (col('country')==col('location')) & (year(col('Datetime'))==year(col('date'))) ),'left')
happiness_CovidStatus = happiness_CovidStatus.selectExpr(happiness_CovidStatus_col)
happiness_CovidStatus = happiness_CovidStatus.dropDuplicates()
happiness_CovidStatus = happiness_CovidStatus.withColumn('ID',concat('date','iso_code'))
happiness_CovidStatus.limit(10).toPandas()

#### After insert into every table , then write all table in Project_Warehouse folder in parquet format

In [ ]:
# Write every single tables in formatted schema and determine how to partition each table

country_covidStatus_table.write.mode('overwrite').partitionBy('iso_code').parquet(path=output_path+'country_covidStatus_table')
country_vacInfoManufacturer_table.write.mode('overwrite').partitionBy('location').parquet(path=output_path+'country_vacInfoManufacturer_table')
vacAdverse_vacInfo_table.write.mode('overwrite').partitionBy('VAX_MANU').parquet(path=output_path+'vacAdverse_vacInfo_table')
vacAdverse_patInfo_table.write.mode('overwrite').partitionBy('VAX_DATE').parquet(path=output_path+'vacAdverse_patInfo_table')
vacAdverse_symInfo_table.write.mode('overwrite').parquet(path=output_path+'vacAdverse_symInfo_table')
unemployRate_table.write.mode('overwrite').partitionBy('country_code').parquet(path=output_path+'unemployRate_table')
happinessRate_table.write.mode('overwrite').partitionBy('country').parquet(path=output_path+'happinessRate_table')
unemploy_CovidStatus.write.mode('overwrite').partitionBy('iso_code').parquet(path=output_path+'unemploy_CovidStatus')
happiness_CovidStatus.write.mode('overwrite').partitionBy('iso_code').parquet(path=output_path+'happiness_CovidStatus')

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Check reading all data data that stored in parquet format from warehouse

country_covidStatus_table = spark.read.parquet(output_path+"country_covidStatus_table")
country_vacInfoManufacturer_table = spark.read.parquet(output_path+"country_vacInfoManufacturer_table")
vacAdverse_vacInfo_table = spark.read.parquet(output_path+"vacAdverse_vacInfo_table")
vacAdverse_patInfo_table = spark.read.parquet(output_path+"vacAdverse_patInfo_table")
vacAdverse_symInfo_table = spark.read.parquet(output_path+"vacAdverse_symInfo_table")
unemployRate_table = spark.read.parquet(output_path+"unemployRate_table")
happinessRate_table = spark.read.parquet(output_path+"happinessRate_table")
unemploy_CovidStatus = spark.read.parquet(output_path+"unemploy_CovidStatus")
happiness_CovidStatus = spark.read.parquet(output_path+"happiness_CovidStatus")

In [ ]:
# Do the count test in every table

if (country_covidStatus_table.count() < 100):
    print('country_covidStatus_table count test not passed')
if (country_vacInfoManufacturer_table.count() < 100):
    print('country_vacInfoManufacturer_table count test not passed')
if (vacAdverse_vacInfo_table.count() < 100):
    print('vacAdverse_vacInfo_table count test not passed')
if (vacAdverse_patInfo_table.count() < 100):
    print('vacAdverse_patInfo_table count test not passed')
if (vacAdverse_symInfo_table.count() < 100):
    print('vacAdverse_symInfo_table count test not passed')
if (unemployRate_table.count() < 100):
    print('unemployRate_table count test not passed')
if (happinessRate_table.count() < 100):
    print('happinessRate_table count test not passed')
if (unemploy_CovidStatus.count() < 100):
    print('unemploy_CovidStatus count test not passed')
if (unemploy_CovidStatus.count() < 100):
    print('unemploy_CovidStatus count test not passed')
print('COUNT TESTS ALL PASSED!')

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Example for user to do the queries

* If user wish to know the consistency between unemployment rate and COVID-19 situation of Austria, then user can run the following query.


In [ ]:
unemploy_CovidStatus.createOrReplaceTempView('unemployAndCovidStatus')
spark.sql("""SELECT * FROM unemployAndCovidStatus WHERE iso_code = 'AUS' ORDER BY date ASC""").show()

* If user wish to know the consistency between happiness rate and COVID-19 situation of Haiti, then user can run the following query.

In [ ]:
happiness_CovidStatus.createOrReplaceTempView('happinessAndCovidStatus')
spark.sql("""SELECT * FROM happinessAndCovidStatus WHERE iso_code = 'HTI' ORDER BY date ASC""").show()

_________________


#### Step 5: Complete Project Write Up
#### choice of tools and technologies for the project

        In this project I used Spark to visualize data because it's easier to do schema on read and I also use Spark to clean the data as well, At a later stage, I recommend using Spark to process the data with the better environment and support larger dataset such as Amazon Elastic Map Reduce (EMR). Also, to perform automated updates, I recommend integrating the ETL pipeline into an Airflow DAG.
    
        The Jupyter Notebook is used to show the steps how I structured the project and easier to markdown and explain in each step about what I did. Apart from this, Python is an often used programming language and was used because it is the language I am the most comfortable with.
    
    
#### how often the data should be updated and why

        The COVID-19 situation still continues to occur, and the vaccination and other informations are also necessary to assess and deal with such situations. thus, In my opinion. monthly update is what I recommended the most.
        
        
#### How I would handle the following sceanarios :
 * The data was increased by 100x.
     * Use Spark to process the data efficiently in a distributed way e.g. with EMR.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
     * Use Airflow and create a DAG that performs the logic of the described pipeline.
 * The database needed to be accessed by 100+ people.
     * Use RedShift to have the data stored in a way that it can efficiently be accessed by many people.

_________________